In [2]:
!pip install selenium

     --------------------------------------- 10.2/10.2 MB 27.1 MB/s eta 0:00:00
     ------------------------------------- 448.3/448.3 kB 13.7 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0


In [14]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service

In [51]:
# first link scraped: https://www.youtube.com/watch?v=kuhhT_cBtFU&t=2s
# Second link scraped: https://www.youtube.com/watch?v=v3abZ4aAGUU

In [52]:
service = Service(executable_path=r'C:\Users\Guillaume\Desktop\chromedriver-win64\chromedriver.exe')
data = []
youtube_video_url = "https://www.youtube.com/watch?v=kuhhT_cBtFU&t=2s"

with webdriver.Chrome(service=service) as driver:
    wait = WebDriverWait(driver, 15)
    driver.get(youtube_video_url)

    # Scroll to load comments
    for item in range(200):  # Adjust range to load more comments
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(2)  

    # Collect comment data
    comments = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ytd-comment-thread-renderer")))
    
    for comment in comments:
        # Username
        try:
            author = comment.find_element(By.XPATH, ".//span[contains(text(),'@')]").text.strip()
        except NoSuchElementException:  # In case there is no username found
            author = "Unknown"

        # Comment text
        try:
            comment_text = comment.find_element(By.ID, 'content-text').text.strip()
        except NoSuchElementException:  # In case there is no text found
            comment_text = "No Text"

        # Comment date
        try:
            comment_date = comment.find_element(By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.yt-formatted-string").text.strip()
        except NoSuchElementException:  # In case there is no date found
            comment_date = "Unknown Date"

        # Likes
        try:
            comment_likes = comment.find_element(By.ID, 'vote-count-middle').get_attribute('aria-label')
        except NoSuchElementException:  # In case there is no like count found
            comment_likes = '0'

        # Aggregate data in a dictionary and append to list
        data.append({
            'username': author,
            'comment': comment_text,
            'date': comment_date,
            'likes': comment_likes,
        })
        
        

In [53]:
# Create DataFrame and save to CSV
df = pd.DataFrame(data, columns=['username', 'comment', 'date', 'likes'])

# Display the first few rows of the DataFrame
df

,username,comment,date,likes
0,@jackjolly789,I'm not from the US but I sure as hell know no...,il y a 3 ans,"160 ""J'aime"""
1,@DenverEight,"I remember in Captain Planet, Mati’s power of ...",il y a 3 ans,None
2,@jhartmggc,It’s tragic how this all unfolded. Unfortunate...,il y a 3 ans,None
3,@datofficial6062,"We dont need his ""help to understand what we'r...",il y a 3 ans,"289 ""J'aime"""
4,@panthercub10,You know it odd but the more this happens the ...,il y a 3 ans,"1 mention ""J'aime"""
...,...,...,...,...
1017,@brainwashingdetergent4128,Trash CNN,il y a 3 ans,None
1018,@RobertLindeman-th7ff,NB: Don't steal police weapons and attempt to...,il y a 3 ans (modifié),"3 ""J'aime"""
1019,@justsomeponywithamustache8731,Faker than the Last of Us 2 reviews,il y a 3 ans,None
1020,@williamliles7647,The dumbass was grabbing for his gun belt.Resi...,il y a 3 ans,None


In [58]:
# Save to CSV file
df.to_csv('youtube_comments.csv', index="username")

In [54]:
unique_counts = df['date'].value_counts()

# Display the unique values and their counts
print(unique_counts)

il y a 3 ans              937
il y a 3 ans (modifié)     80
il y a 2 ans                2
il y a 10 mois              1
il y a 1 an                 1
il y a 1 mois               1
Name: date, dtype: int64


In [55]:
unique_counts = df['likes'].value_counts()

# Display the unique values and their counts
print(unique_counts)

1 mention "J'aime"    95
6 "J'aime"            26
2 "J'aime"            20
7 "J'aime"            19
5 "J'aime"            14
                      ..
99 "J'aime"            1
219 "J'aime"           1
227 "J'aime"           1
223 "J'aime"           1
151 "J'aime"           1
Name: likes, Length: 82, dtype: int64


In [56]:
# Define a function to standardize the date text
def standardize_date(text):
    # Replace years and variants
    text = pd.Series(text).str.replace(r'il y a (\d+) ans', r'\1 years ago', regex=True)
    text = text.str.replace(r'il y a 1 an', r'1 year ago', regex=False)  # Singular year
    # Replace months and variants
    text = text.str.replace(r'il y a (\d+) mois', r'\1 months ago', regex=True)
    # Replace weeks, days, and hours
    text = text.str.replace(r'il y a (\d+) semaine', r'\1 weeks ago', regex=True)
    text = text.str.replace(r'il y a (\d+) jour', r'\1 days ago', regex=True)
    text = text.str.replace(r'il y a (\d+) heure', r'\1 hours ago', regex=True)
    # Handle modified dates, assuming "(modifié)" comes after the time indication
    text = text.str.replace(' (modifié)', '', regex=False)
    return text

# Apply the function to the 'date' column
df['date'] = standardize_date(df['date'])

# Print out unique value counts for dates
unique_dates = df['date'].value_counts()
print(unique_dates)

# Define a function to clean up likes to keep only the number
def clean_likes(likes_series):
    # Extract numbers and convert to integers, fill NaNs with zero
    return likes_series.str.extract('(\d+)')[0].fillna(0).astype(int)

# Apply the function to the 'likes' column
df['likes'] = clean_likes(df['likes'])


3 years ago      1017
2 years ago         2
10 months ago       1
1 year ago          1
1 months ago        1
Name: date, dtype: int64


In [57]:
# Now, the DataFrame should have 'date' in the format "X years ago" and 'likes' as integers
df.head()

,username,comment,date,likes
0,@jackjolly789,I'm not from the US but I sure as hell know no...,3 years ago,160
1,@DenverEight,"I remember in Captain Planet, Mati’s power of ...",3 years ago,0
2,@jhartmggc,It’s tragic how this all unfolded. Unfortunate...,3 years ago,0
3,@datofficial6062,"We dont need his ""help to understand what we'r...",3 years ago,289
4,@panthercub10,You know it odd but the more this happens the ...,3 years ago,1


In [59]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('youtube_comments_cleaned.csv', index=False)

In [62]:
# Load the first CSV file into a DataFrame
df1 = pd.read_csv('youtube_comments_cleaned.csv')

# Load the second CSV file into a DataFrame
df2 = pd.read_csv('youtube_comments_cleaned2.csv')

# Concatenate the two DataFrames, one after the other
scraped_df = pd.concat([df1, df2], ignore_index=True)

In [65]:
# Save the concatenated DataFrame to a new CSV file
scraped_df.to_csv('scraped_df.csv', index=False)

In [66]:
scraped_df

,username,comment,date,likes
0,@jackjolly789,I'm not from the US but I sure as hell know no...,3 years ago,160
1,@DenverEight,"I remember in Captain Planet, Mati’s power of ...",3 years ago,0
2,@jhartmggc,It’s tragic how this all unfolded. Unfortunate...,3 years ago,0
3,@datofficial6062,"We dont need his ""help to understand what we'r...",3 years ago,289
4,@panthercub10,You know it odd but the more this happens the ...,3 years ago,1
...,...,...,...,...
2233,@claudiabotto3048,Trump,10 months ago,0
2234,@MichelSantana-xg5qi,Trump and your beautiful lie.,5 months ago,0
2235,@slifer0081,As much as I hate trump and love migrants I ha...,2 months ago,0
2236,@george4821,Trump was the rude one here.,3 months ago,0
